In [ ]:
import cv2
import dlib
import time
import threading
import math
import csv
import numpy as np
from lane_function import lane

carCascade = cv2.CascadeClassifier('myhaar.xml')
video_path = 'CutPOV.mp4'

WIDTH = 1280
HEIGHT = 720


def estimateSpeed(location1, location2):
    d_pixels = math.sqrt(math.pow(location2[0] - location1[0], 2) + math.pow(location2[1] - location1[1], 2))
    ppm = 8.8
    d_meters = d_pixels / ppm
    fps = 18
    speed = d_meters * fps * 3.6
    return speed

def trackMultipleObjects(video):
    rectangleColor = (0, 255, 0)
    frameCounter = 0
    currentCarID = 0
    fps = 0

    carTracker = {}
    carNumbers = {}
    carLocation1 = {}
    carLocation2 = {}
    speed = [None] * 1000
    
    with open('speed_data.csv', mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Frame', 'Car ID', 'Speed (km/hr)'])
        while True:
            start_time = time.time()
            rc, image = video.read()
            if type(image) == type(None):
                break
            
            image = cv2.resize(image, (WIDTH, HEIGHT))
            resultImage = image.copy()

            frameCounter = frameCounter + 1

            carIDtoDelete = []

            for carID in carTracker.keys():
                trackingQuality = carTracker[carID].update(image)

                if trackingQuality < 7:
                    carIDtoDelete.append(carID)

            for carID in carIDtoDelete:
                print('Removing carID ' + str(carID) + ' from list of trackers.')
                print('Removing carID ' + str(carID) + ' previous location.')
                print('Removing carID ' + str(carID) + ' current location.')
                carTracker.pop(carID, None)
                carLocation1.pop(carID, None)
                carLocation2.pop(carID, None)

            if not (frameCounter % 10):
                gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
                cars = carCascade.detectMultiScale(gray, 1.1, 13, 18, (24, 24))

                for (_x, _y, _w, _h) in cars:
                    x = int(_x)
                    y = int(_y)
                    w = int(_w)
                    h = int(_h)

                    x_bar = x + 0.5 * w
                    y_bar = y + 0.5 * h

                    matchCarID = None

                    for carID in carTracker.keys():
                        trackedPosition = carTracker[carID].get_position()

                        t_x = int(trackedPosition.left())
                        t_y = int(trackedPosition.top())
                        t_w = int(trackedPosition.width())
                        t_h = int(trackedPosition.height())

                        t_x_bar = t_x + 0.5 * t_w
                        t_y_bar = t_y + 0.5 * t_h

                        if ((t_x <= x_bar <= (t_x + t_w)) and (t_y <= y_bar <= (t_y + t_h)) and (
                                x <= t_x_bar <= (x + w)) and (y <= t_y_bar <= (y + h))):
                            matchCarID = carID

                    if matchCarID is None:
                        print('Creating new tracker ' + str(currentCarID))

                        tracker = dlib.correlation_tracker()
                        tracker.start_track(image, dlib.rectangle(x, y, x + w, y + h))

                        carTracker[currentCarID] = tracker
                        carLocation1[currentCarID] = [x, y, w, h]

                        currentCarID = currentCarID + 1

            for carID in carTracker.keys():
                trackedPosition = carTracker[carID].get_position()

                t_x = int(trackedPosition.left())
                t_y = int(trackedPosition.top())
                t_w = int(trackedPosition.width())
                t_h = int(trackedPosition.height())

                cv2.rectangle(resultImage, (t_x, t_y), (t_x + t_w, t_y + t_h), rectangleColor, 4)

                carLocation2[carID] = [t_x, t_y, t_w, t_h]

            end_time = time.time()
            out = cv2.VideoWriter('outpy.avi', cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'), 10, (WIDTH, HEIGHT))

            if not (end_time == start_time):
                fps = 1.0 / (end_time - start_time)

            for i in carLocation1.keys():
                if frameCounter % 1 == 0:
                    [x1, y1, w1, h1] = carLocation1[i]
                    [x2, y2, w2, h2] = carLocation2[i]

                    carLocation1[i] = [x2, y2, w2, h2]

                    if [x1, y1, w1, h1] != [x2, y2, w2, h2]:
                        if (speed[i] == None or speed[i] == 0) and y1 >= 275 and y1 <= 285:
                            speed[i] = estimateSpeed([x1, y1, w1, h1], [x2, y2, w2, h2])

                        if speed[i] != None and y1 >= 180:
                            cv2.putText(resultImage, str(int(speed[i])) + " km/hr", (int(x1 + w1 / 2), int(y1 - 5)),
                                        cv2.FONT_HERSHEY_SIMPLEX, 0.75, (255, 255, 255), 2)
                            writer.writerow([frameCounter,i,speed[i]])

            # Lane detection
            lower_yellow = np.array([18, 94, 140], dtype=np.uint8)
            upper_yellow = np.array([48, 255, 255], dtype=np.uint8)
            roi_vertices = np.array([[(100, 480), (850, 480), (650, 300), (450, 300)]], dtype=np.int32)

            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            blur = cv2.GaussianBlur(gray, (5, 5), 0)
            edges = cv2.Canny(blur, 50, 150)

            mask = np.zeros_like(edges)
            cv2.fillPoly(mask, roi_vertices, 255)
            masked_edges = cv2.bitwise_and(edges, mask)

            lines = cv2.HoughLinesP(masked_edges, 1, np.pi / 180, 40, np.array([]), 100, 50)
            line_image = np.zeros_like(image)
            if lines is not None:
                for line in lines:
                    x1, y1, x2, y2 = line[0]
                    cv2.line(line_image, (x1, y1), (x2, y2), (0, 0, 255), 5)

            merged_image = cv2.addWeighted(image, 0.8, line_image, 1, 0)

            cv2.imshow('result', np.hstack((resultImage, merged_image)))

            if cv2.waitKey(33) == 27:
                break

    cv2.destroyAllWindows()

if __name__ == '__main__':
    video = cv2.VideoCapture(video_path)
    trackMultipleObjects(video)
    video.release()  # Release the video capture object

Creating new tracker 0
Creating new tracker 1
Creating new tracker 2
Creating new tracker 3
Creating new tracker 4
Removing carID 4 from list of trackers.
Removing carID 4 previous location.
Removing carID 4 current location.
Removing carID 2 from list of trackers.
Removing carID 2 previous location.
Removing carID 2 current location.
Removing carID 3 from list of trackers.
Removing carID 3 previous location.
Removing carID 3 current location.
Creating new tracker 5
Creating new tracker 6
Creating new tracker 7
Creating new tracker 8
Creating new tracker 9
Removing carID 7 from list of trackers.
Removing carID 7 previous location.
Removing carID 7 current location.
Removing carID 8 from list of trackers.
Removing carID 8 previous location.
Removing carID 8 current location.
Creating new tracker 10
Removing carID 9 from list of trackers.
Removing carID 9 previous location.
Removing carID 9 current location.
Removing carID 10 from list of trackers.
Removing carID 10 previous location.
Re

In [1]:
import cv2
import dlib
import time
import threading
import math
import csv
import numpy as np
from lane_function import lane

carCascade = cv2.CascadeClassifier('myhaar.xml')
video_path = '/Users/fysiki_mac/Desktop/SIG/bus.mp4'

WIDTH = 1280
HEIGHT = 720


def estimateSpeed(location1, location2):
    d_pixels = math.sqrt(math.pow(location2[0] - location1[0], 2) + math.pow(location2[1] - location1[1], 2))
    ppm = 8.8
    d_meters = d_pixels / ppm
    fps = 18
    speed = d_meters * fps * 3.6
    return speed

def trackMultipleObjects(video):
    rectangleColor = (0, 255, 0)
    frameCounter = 0
    currentCarID = 0
    fps = 0

    carTracker = {}
    carNumbers = {}
    carLocation1 = {}
    carLocation2 = {}
    speed = [None] * 1000
    
    with open('speed_data.csv', mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Frame', 'Car ID', 'Speed (km/hr)','Position (X)','Position (Y)'])
        while True:
            start_time = time.time()
            rc, image = video.read()
            if type(image) == type(None):
                break
            
            image = cv2.resize(image, (WIDTH, HEIGHT))
            resultImage = image.copy()

            frameCounter = frameCounter + 1

            carIDtoDelete = []

            for carID in carTracker.keys():
                trackingQuality = carTracker[carID].update(image)

                if trackingQuality < 7:
                    carIDtoDelete.append(carID)

            for carID in carIDtoDelete:
                print('Removing carID ' + str(carID) + ' from list of trackers.')
                print('Removing carID ' + str(carID) + ' previous location.')
                print('Removing carID ' + str(carID) + ' current location.')
                carTracker.pop(carID, None)
                carLocation1.pop(carID, None)
                carLocation2.pop(carID, None)

            if not (frameCounter % 10):
                gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
                cars = carCascade.detectMultiScale(gray, 1.1, 13, 18, (24, 24))

                for (_x, _y, _w, _h) in cars:
                    x = int(_x)
                    y = int(_y)
                    w = int(_w)
                    h = int(_h)

                    x_bar = x + 0.5 * w
                    y_bar = y + 0.5 * h

                    matchCarID = None

                    for carID in carTracker.keys():
                        trackedPosition = carTracker[carID].get_position()

                        t_x = int(trackedPosition.left())
                        t_y = int(trackedPosition.top())
                        t_w = int(trackedPosition.width())
                        t_h = int(trackedPosition.height())

                        t_x_bar = t_x + 0.5 * t_w
                        t_y_bar = t_y + 0.5 * t_h

                        if ((t_x <= x_bar <= (t_x + t_w)) and (t_y <= y_bar <= (t_y + t_h)) and (
                                x <= t_x_bar <= (x + w)) and (y <= t_y_bar <= (y + h))):
                            matchCarID = carID

                    if matchCarID is None:
                        print('Creating new tracker ' + str(currentCarID))

                        tracker = dlib.correlation_tracker()
                        tracker.start_track(image, dlib.rectangle(x, y, x + w, y + h))

                        carTracker[currentCarID] = tracker
                        carLocation1[currentCarID] = [x, y, w, h]

                        currentCarID = currentCarID + 1

            for carID in carTracker.keys():
                trackedPosition = carTracker[carID].get_position()

                t_x = int(trackedPosition.left())
                t_y = int(trackedPosition.top())
                t_w = int(trackedPosition.width())
                t_h = int(trackedPosition.height())

                cv2.rectangle(resultImage, (t_x, t_y), (t_x + t_w, t_y + t_h), rectangleColor, 4)

                carLocation2[carID] = [t_x, t_y, t_w, t_h]

            end_time = time.time()
            out = cv2.VideoWriter('outpy.avi', cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'), 10, (WIDTH, HEIGHT))

            if not (end_time == start_time):
                fps = 1.0 / (end_time - start_time)

            for i in carLocation1.keys():
                if frameCounter % 1 == 0:
                    [x1, y1, w1, h1] = carLocation1[i]
                    [x2, y2, w2, h2] = carLocation2[i]

                    carLocation1[i] = [x2, y2, w2, h2]

                    if [x1, y1, w1, h1] != [x2, y2, w2, h2]:
                        if (speed[i] == None or speed[i] == 0) and y1 >= 275 and y1 <= 285:
                            speed[i] = estimateSpeed([x1, y1, w1, h1], [x2, y2, w2, h2])

                        if speed[i] != None and y1 >= 180:
                            cv2.putText(resultImage, str(int(speed[i])) + " km/hr", (int(x1 + w1 / 2), int(y1 - 5)),
                                        cv2.FONT_HERSHEY_SIMPLEX, 0.75, (255, 255, 255), 2)
                            writer.writerow([frameCounter,i,speed[i],x_bar,y_bar])

            # Lane detection
            lower_yellow = np.array([18, 94, 140], dtype=np.uint8)
            upper_yellow = np.array([48, 255, 255], dtype=np.uint8)
            roi_vertices = np.array([[(100, 480), (850, 480), (650, 300), (450, 300)]], dtype=np.int32)

            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            blur = cv2.GaussianBlur(gray, (5, 5), 0)
            edges = cv2.Canny(blur, 50, 150)

            mask = np.zeros_like(edges)
            cv2.fillPoly(mask, roi_vertices, 255)
            masked_edges = cv2.bitwise_and(edges, mask)

            lines = cv2.HoughLinesP(masked_edges, 1, np.pi / 180, 40, np.array([]), 100, 50)
            line_image = np.zeros_like(image)
            if lines is not None:
                for line in lines:
                    x1, y1, x2, y2 = line[0]
                    cv2.line(line_image, (x1, y1), (x2, y2), (0, 0, 255), 5)

            merged_image = cv2.addWeighted(image, 0.8, line_image, 1, 0)

            cv2.imshow('result', np.hstack((resultImage, merged_image)))

            if cv2.waitKey(33) == 27:
                break

    cv2.destroyAllWindows()

if __name__ == '__main__':
    video = cv2.VideoCapture(video_path)
    trackMultipleObjects(video)
    video.release()  # Release the video capture object

Creating new tracker 0
Creating new tracker 1
Removing carID 1 from list of trackers.
Removing carID 1 previous location.
Removing carID 1 current location.
Creating new tracker 2
Removing carID 2 from list of trackers.
Removing carID 2 previous location.
Removing carID 2 current location.
Creating new tracker 3
Creating new tracker 4
Removing carID 3 from list of trackers.
Removing carID 3 previous location.
Removing carID 3 current location.
Removing carID 4 from list of trackers.
Removing carID 4 previous location.
Removing carID 4 current location.
Creating new tracker 5
Creating new tracker 6
Creating new tracker 7
Removing carID 7 from list of trackers.
Removing carID 7 previous location.
Removing carID 7 current location.
Creating new tracker 8
Creating new tracker 9
Removing carID 8 from list of trackers.
Removing carID 8 previous location.
Removing carID 8 current location.
Creating new tracker 10
Creating new tracker 11
Creating new tracker 12
Creating new tracker 13
Removing

In [ ]:
import cv2
import dlib
import time
import threading
import math
import csv
import numpy as np
from lane_function import lane

carCascade = cv2.CascadeClassifier('myhaar.xml')
video_path = 'CutPOV.mp4'

WIDTH = 1280
HEIGHT = 720


def estimateSpeed(location1, location2):
    d_pixels = math.sqrt(math.pow(location2[0] - location1[0], 2) + math.pow(location2[1] - location1[1], 2))
    ppm = 8.8
    d_meters = d_pixels / ppm
    fps = 18
    speed = d_meters * fps * 3.6
    return speed

def trackMultipleObjects(video):
    rectangleColor = (0, 255, 0)
    frameCounter = 0
    currentCarID = 0
    fps = 0

    carTracker = {}
    carNumbers = {}
    carLocation1 = {}
    carLocation2 = {}
    speed = [None] * 1000
    
    with open('speed_data.csv', mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Frame', 'Car ID', 'Speed (km/hr)','Position (X)','Position (Y)'])
        while True:
            start_time = time.time()
            rc, image = video.read()
            if type(image) == type(None):
                break
            
            image = cv2.resize(image, (WIDTH, HEIGHT))
            resultImage = image.copy()

            frameCounter = frameCounter + 1

            carIDtoDelete = []

            for carID in carTracker.keys():
                trackingQuality = carTracker[carID].update(image)

                if trackingQuality < 7:
                    carIDtoDelete.append(carID)

            for carID in carIDtoDelete:
                print('Removing carID ' + str(carID) + ' from list of trackers.')
                print('Removing carID ' + str(carID) + ' previous location.')
                print('Removing carID ' + str(carID) + ' current location.')
                carTracker.pop(carID, None)
                carLocation1.pop(carID, None)
                carLocation2.pop(carID, None)

            if not (frameCounter % 10):
                gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
                cars = carCascade.detectMultiScale(gray, 1.1, 13, 18, (24, 24))

                for (_x, _y, _w, _h) in cars:
                    x = int(_x)
                    y = int(_y)
                    w = int(_w)
                    h = int(_h)

                    x_bar = x + 0.5 * w
                    y_bar = y + 0.5 * h

                    matchCarID = None

                    for carID in carTracker.keys():
                        trackedPosition = carTracker[carID].get_position()

                        t_x = int(trackedPosition.left())
                        t_y = int(trackedPosition.top())
                        t_w = int(trackedPosition.width())
                        t_h = int(trackedPosition.height())

                        t_x_bar = t_x + 0.5 * t_w
                        t_y_bar = t_y + 0.5 * t_h

                        if ((t_x <= x_bar <= (t_x + t_w)) and (t_y <= y_bar <= (t_y + t_h)) and (
                                x <= t_x_bar <= (x + w)) and (y <= t_y_bar <= (y + h))):
                            matchCarID = carID

                    if matchCarID is None:
                        print('Creating new tracker ' + str(currentCarID))

                        tracker = dlib.correlation_tracker()
                        tracker.start_track(image, dlib.rectangle(x, y, x + w, y + h))

                        carTracker[currentCarID] = tracker
                        carLocation1[currentCarID] = [x, y, w, h]

                        currentCarID = currentCarID + 1

            for carID in carTracker.keys():
                trackedPosition = carTracker[carID].get_position()

                t_x = int(trackedPosition.left())
                t_y = int(trackedPosition.top())
                t_w = int(trackedPosition.width())
                t_h = int(trackedPosition.height())

                cv2.rectangle(resultImage, (t_x, t_y), (t_x + t_w, t_y + t_h), rectangleColor, 4)

                carLocation2[carID] = [t_x, t_y, t_w, t_h]

            end_time = time.time()
            out = cv2.VideoWriter('outpy.avi', cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'), 10, (WIDTH, HEIGHT))

            if not (end_time == start_time):
                fps = 1.0 / (end_time - start_time)

            for i in carLocation1.keys():
                if frameCounter % 1 == 0:
                    [x1, y1, w1, h1] = carLocation1[i]
                    [x2, y2, w2, h2] = carLocation2[i]

                    carLocation1[i] = [x2, y2, w2, h2]

                    if [x1, y1, w1, h1] != [x2, y2, w2, h2]:
                        if (speed[i] == None or speed[i] == 0) and y1 >= 275 and y1 <= 285:
                            speed[i] = estimateSpeed([x1, y1, w1, h1], [x2, y2, w2, h2])

                        if speed[i] != None and y1 >= 180:
                            cv2.putText(resultImage, str(int(speed[i])) + " km/hr", (int(x1 + w1 / 2), int(y1 - 5)),
                                        cv2.FONT_HERSHEY_SIMPLEX, 0.75, (255, 255, 255), 2)
                            writer.writerow([frameCounter,i,speed[i],x_bar,y_bar])

            # Lane detection
            lower_yellow = np.array([18, 94, 140], dtype=np.uint8)
            upper_yellow = np.array([48, 255, 255], dtype=np.uint8)
            roi_vertices = np.array([[(300, 580), (880, 580), (700, 400), (550, 400)]], dtype=np.int32)

            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            blur = cv2.GaussianBlur(gray, (5, 5), 0)
            edges = cv2.Canny(blur, 50, 150)

            mask = np.zeros_like(edges)
            cv2.fillPoly(mask, roi_vertices, 255)
            masked_edges = cv2.bitwise_and(edges, mask)

            lines = cv2.HoughLinesP(masked_edges, 1, np.pi / 180, 40, np.array([]), 100, 50)
            line_image = np.zeros_like(image)
            if lines is not None:
                for line in lines:
                    x1, y1, x2, y2 = line[0]
                    cv2.line(line_image, (x1, y1), (x2, y2), (0, 0, 255), 5)

            merged_image = cv2.addWeighted(image, 0.8, line_image, 1, 0)


            cv2.imshow('result', np.hstack((resultImage, merged_image)))

            if cv2.waitKey(33) == 27:
                break

    cv2.destroyAllWindows()

if __name__ == '__main__':
    video = cv2.VideoCapture(video_path)
    trackMultipleObjects(video)
    video.release()  # Release the video capture object

Creating new tracker 0
Creating new tracker 1
Creating new tracker 2
Creating new tracker 3
Creating new tracker 4
Removing carID 4 from list of trackers.
Removing carID 4 previous location.
Removing carID 4 current location.
Removing carID 2 from list of trackers.
Removing carID 2 previous location.
Removing carID 2 current location.
Removing carID 3 from list of trackers.
Removing carID 3 previous location.
Removing carID 3 current location.
Creating new tracker 5
Creating new tracker 6
Creating new tracker 7
Creating new tracker 8
Creating new tracker 9
Removing carID 7 from list of trackers.
Removing carID 7 previous location.
Removing carID 7 current location.
Removing carID 8 from list of trackers.
Removing carID 8 previous location.
Removing carID 8 current location.
Creating new tracker 10
Removing carID 9 from list of trackers.
Removing carID 9 previous location.
Removing carID 9 current location.
Removing carID 10 from list of trackers.
Removing carID 10 previous location.
Re